# Analyse exploratoire du marché Airbnb - Oslo
**Auteur :** Data Analyst (Assistant IA) & Business Analyst
**Contexte :** Projet Master 2

## 1. Introduction & Contexte métier
Ce notebook présente une **analyse exploratoire de données (EDA)** approfondie sur le marché Airbnb à Oslo. Il a pour but de fournir des insights clés à un investisseur potentiel ou à une collectivité locale.

Les **objectifs principaux** sont :
1. Comprendre la **structure de l'offre** (quartiers, types de logements).
2. Analyser les **prix** et identifier les zones les plus valorisées.
3. Étudier la **concurrence** (hôtes professionnels vs particuliers).
4. Estimer la **demande** via les avis et la disponibilité.
5. Proposer une **segmentation** du marché.

---
*Disclaimer : Cette analyse est basée sur une extraction statique (snapshot) et comporte des limites inhérentes à la nature des données déclaratives et publiques d'Airbnb.*


## 2. Chargement des données
Nous commençons par importer les librairies nécessaires et charger le dataset.
Nous utilisons `pandas` pour la manipulation, et `matplotlib`/`seaborn` pour la visualisation.

Paramètres graphiques : nous définissons un style lisible et une taille de figure par défaut.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

# Configuration du style des graphiques
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['axes.labelsize'] = 14


Chargement du fichier `listings.csv`. Si le fichier n'est pas trouvé, une erreur sera levée demandant de vérifier le chemin.


In [ ]:
try:
    df_raw = pd.read_csv('listings.csv')
    print("Données chargées avec succès !")
    print(f"Dimensions du dataset : {df_raw.shape}")
except FileNotFoundError:
    print("ERREUR : Le fichier 'listings.csv' est introuvable. Veuillez vérifier le répertoire de travail.")


Affichage des premières lignes pour vérifier la structure :


In [ ]:
# Aperçu des données brutes
df_raw.head(5)


## 3. Compréhension & Nettoyage des données
Avant d'analyser, nous devons vérifier la qualité des données : types, valeurs manquantes, cohérence.

### 3.1. Infos générales et types


In [ ]:
df_raw.info()


### 3.2. Valeurs manquantes
Identifions les colonnes avec des `NaN`.


In [ ]:
missing_values = df_raw.isnull().sum()
missing_values[missing_values > 0]


**Observations & Actions de nettoyage :**
- `neighbourhood_group`: Souvent vide dans les extractions européennes (comme Oslo), nous utiliserons principalement `neighbourhood`.
- `last_review`, `reviews_per_month`: Les valeurs manquantes signifient généralement **0 reviews**. C'est une information en soi (listing inactif ou nouveau).
- `license`: Souvent vide, information légale. Pas critique pour l'analyse économique.

Nous allons :
1. Convertir `last_review` en format **datetime**.
2. Créer une copie `df` pour le travail nettoyé.
3. Remplacer les NaN de `reviews_per_month` par 0 pour faciliter les calculs statistiques (hypothèse raisonnable).


In [ ]:
# Création de la copie de travail
df = df_raw.copy()

# Conversion de date
if 'last_review' in df.columns:
    df['last_review'] = pd.to_datetime(df['last_review'], errors='coerce')

# Traitement des NaNs pour l'analyse
if 'reviews_per_month' in df.columns:
    df['reviews_per_month'] = df['reviews_per_month'].fillna(0)

# Vérification rapide
df[['last_review', 'reviews_per_month']].info()


### 3.3. Résumé statistique des variables numériques


In [ ]:
df.describe()


## 4. Vue d’ensemble du marché
Nous analysons ici la **taille** et la **structure** du marché Airbnb à Oslo.

### 4.1. Répartition par Quartier (`neighbourhood`)
Où se situent les logements ?


In [ ]:
# Comptage des listings par quartier
nb_counts = df['neighbourhood'].value_counts()

plt.figure(figsize=(12, 6))
sns.barplot(x=nb_counts.index, y=nb_counts.values, palette='viridis')
plt.xticks(rotation=45, ha='right')
plt.title('Nombre de listings par quartier à Oslo')
plt.xlabel('Quartier')
plt.ylabel('Nombre de listings')
plt.show()


### 4.2. Répartition par Type de logement (`room_type`)
Quelle est l'offre dominante ? Logement entier ou chambre privée ?


In [ ]:
room_counts = df['room_type'].value_counts()

plt.figure(figsize=(8, 5))
sns.barplot(x=room_counts.index, y=room_counts.values, palette='magma')
plt.title('Répartition des types de logements')
plt.xlabel('Type de logement')
plt.ylabel('Nombre de listings')
plt.show()


> **Insights Business (Marché) :**
> - **Concentration :** On observe visuellement quels quartiers dominent (souvent Grünerløkka, Gamle Oslo, Frogner). Si un quartier a très peu d'offres, c'est peut-être une opportunité... ou un signe de manque d'attractivité.
> - **Typologie :** Si "Entire home/apt" est majoritaire, le marché est orienté vers la location touristique complète (familles, groupes) plutôt que le "couchsurfing" ou la chambre chez l'habitant.


## 5. Analyse des prix
L'analyse des prix est critique pour le positionnement.

### 5.1. Distribution des prix et Outliers
Commençons par visualiser la distribution brute.


In [ ]:
plt.figure(figsize=(10, 5))
sns.histplot(df['price'], bins=50, kde=True)
plt.title('Distribution des prix (données brutes)')
plt.xlabel('Prix (NOK)')
plt.show()


La distribution est probablement très asymétrique (longue traîne à droite) avec quelques prix extrêmes (luxe ou erreurs).

### 5.2. Stratégie de traitement des Outliers
Nous devons filtrer les valeurs extrêmes pour ne pas fausser les moyennes.
**Choix méthodologique (Validé par l'utilisateur) :**
- Nous utilisons la méthode de l'écart interquartile (**IQR**).
- Bornes : `[Q1 - 1.5*IQR, Q3 + 1.5*IQR]`.
- Les prix au-delà de la borne supérieure sont considérés comme des outliers atypiques pour cette analyse généraliste.


In [ ]:
# Calcul de l'IQR
Q1 = df['price'].quantile(0.25)
Q3 = df['price'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(f"Seuils pour outliers : < {lower_bound:.2f} ou > {upper_bound:.2f}")

# Création de df_clean
df_clean = df[(df['price'] >= lower_bound) & (df['price'] <= upper_bound)].copy()

print(f"Nombre de listings avant : {len(df)}")
print(f"Nombre de listings après filtrage : {len(df_clean)}")
print(f"Listings supprimés : {len(df) - len(df_clean)}")


### 5.3. Analyse des prix sur données nettoyées
Visualisons à nouveau la distribution et les prix par quartier.


In [ ]:
plt.figure(figsize=(10, 5))
sns.histplot(df_clean['price'], bins=50, kde=True, color='green')
plt.title('Distribution des prix (sans outliers)')
plt.xlabel('Prix (NOK)')
plt.show()


In [ ]:
# Prix par quartier
plt.figure(figsize=(14, 7))
# Tri par prix médian décroissant
order_neigh = df_clean.groupby('neighbourhood')['price'].median().sort_values(ascending=False).index

sns.boxplot(data=df_clean, x='neighbourhood', y='price', order=order_neigh, palette='coolwarm')
plt.xticks(rotation=45, ha='right')
plt.title('Distribution des prix par quartier')
plt.show()


> **Insights Business (Prix) :**
> - Le graphique boxplot permet d'identifier immédiatement les **quartiers "Premium"** (médiane élevée) vs les quartiers **"Abordables"**.
> - La dispersion (taille de la boîte) indique l'hétérogénéité de l'offre dans un quartier.


## 6. Analyse des hôtes & des avis
Nous cherchons à identifier la professionnalisation du marché.


### 6.1. Superhosts
*(Note : La colonne `host_is_superhost` n'étant pas présente dans ce fichier simplifié, nous ne pouvons pas analyser ce statut spécifique. Nous nous concentrons sur les multi-listings.)*


### 6.2. Hôtes professionnels vs particuliers
Un indicateur clé est `calculated_host_listings_count`.
**Hypothèse (Validée) :** Un hôte ayant **3 annonces ou plus** est considéré comme un indicateur de "professionnalisation" ou d'investissement locatif.


In [ ]:
threshold_pro = 3
df_clean['is_pro'] = df_clean['calculated_host_listings_count'] >= threshold_pro

pro_counts = df_clean['is_pro'].value_counts(normalize=True) * 100
print("Répartition des hôtes :")
print(pro_counts)

# Visualisation
plt.figure(figsize=(6, 6))
pro_counts.plot.pie(autopct='%1.1f%%', labels=['Particuliers (<3)', 'Pros (>=3)'], startangle=90, colors=['skyblue', 'salmon'])
plt.title('Estimation de la part des hôtes "Professionnels"')
plt.ylabel('')
plt.show()


In [ ]:
# Impact sur le prix ?
mean_price_pro = df_clean.groupby('is_pro')['price'].mean()
print("Prix moyen par type d'hôte :")
print(mean_price_pro)


> **Insights Business (Hôtes) :**
> - Une part élevée de pros indique un marché mature et commercial.
> - Si les pros sont plus chers, cela peut refléter un service standardisé ou des biens mieux localisés.


## 7. Indicateurs de demande
Nous utilisons `number_of_reviews` et `availability_365` comme proxies (imparfaits) de la demande.
- **Disponibilité faible** (~0) = Potentiellement très demandé OU fermé.
- **Beaucoup d'avis** = Logement très fréquenté.

Visualisons le nombre moyen d'avis par quartier pour voir les zones les plus dynamiques.


In [ ]:
reviews_by_neigh = df_clean.groupby('neighbourhood')['number_of_reviews'].mean().sort_values(ascending=False)

plt.figure(figsize=(12, 6))
sns.barplot(x=reviews_by_neigh.index, y=reviews_by_neigh.values, palette='Blues_r')
plt.xticks(rotation=45, ha='right')
plt.title('Nombre moyen d\'avis par annonce (Indicateur de volume de fréquentation)')
plt.xlabel('Quartier')
plt.ylabel('Moyenne de reviews')
plt.show()


> **Insights Business (Demande) :**
> - Les quartiers avec le plus d'avis par listing sont souvent les cœurs touristiques.
> - Attention au biais : les vieux listings ont accumulé plus d'avis.


## 8. Segmentation (Budget / Mid / Luxe)
Pour mieux cibler les investissements, nous segmentons le marché en 3 catégories basées sur le prix.

**Méthodologie (Validée) :** Utilisation des **terciles** (33% / 33% / 33%) sur `df_clean`.
- **Budget** : < 33ème percentile
- **Mid-Range** : entre 33% et 66%
- **Luxe** : > 66ème percentile


In [ ]:
p33 = df_clean['price'].quantile(0.33)
p66 = df_clean['price'].quantile(0.66)

def segment_price(price):
    if price < p33:
        return 'Budget'
    elif price < p66:
        return 'Mid-Range'
    else:
        return 'Luxe'

df_clean['segment'] = df_clean['price'].apply(segment_price)

print(f"Seuil Budget : < {p33:.0f} NOK")
print(f"Seuil Luxe   : > {p66:.0f} NOK")


Regardons la répartition de ces segments par quartier pour identifier le "profil" de chaque quartier.


In [ ]:
# Tableau croisé
ct = pd.crosstab(df_clean['neighbourhood'], df_clean['segment'])
# Normalisation pour voir les %
ct_norm = ct.div(ct.sum(1), axis=0)

ct_norm.plot(kind='bar', stacked=True, figsize=(12, 7), colormap='viridis')
plt.title('Segmentation de gamme par quartier')
plt.xlabel('Quartier')
plt.ylabel('Proportion')
plt.legend(title='Segment', loc='upper left', bbox_to_anchor=(1, 1))
plt.tight_layout()
plt.show()


> **Insights Business (Segmentation) :**
> - Ce graphique est crucial : il montre si un quartier est exclusif (dominé par le Luxe) ou accessible.
> - Pour un investisseur, entrer avec une offre "Luxe" dans un quartier "Budget" peut être risqué mais aussi différenciant.


## 9. Synthèse business & limites de l’analyse

### 9.1. Principaux Insights Airbnb Oslo
1. **Géographie** : Le marché est concentré dans quelques quartiers clés. (Se référer au graphe 4.1).
2. **Prix** : Il existe une disparité nette. Les quartiers de l'ouest (souvent Frogner/Majorstuen) tendent à être plus chers (Luxe) que ceux de l'est (Gamle Oslo).
3. **Professionnalisation** : La part d'hôtes multi-listings (pro) permet d'évaluer la saturation concurrentielle.
4. **Opportunités** : La segmentation montre que certains quartiers sont peut-être sous-exploités sur le segment "Mid-Range".

### 9.2. Limites et Biais
- **Données déclaratives** : Les prix affichés ne sont pas forcément les prix finaux payés (frais de ménage, promos, négociation).
- **Disponibilité** : Un calendrier bloqué (`availability_365 = 0`) ne veut pas dire "complet", cela peut dire "fermé".
- **Absence de données temporelles** : Nous analysons un instantané (snapshot). Impossible de voir la saisonnalité réelle (été/hiver) sans données historiques.
- **Biais du survivant** : Seuls les listings actifs ou présents lors du scrap sont visibles.


## 10. Fichiers à rendre (projet individuel)

Les fichiers produits dans le cadre de ce travail individuel sont :
1. **`oslo_airbnb_EDA.ipynb`** : Ce notebook complet.
2. **`interaction_IA_oslo.md`** (ou PDF) : Le journal des échanges avec l'assistant IA.

*(Rappel : La modélisation prédictive / Régression du prix sera traitée dans la partie groupe.)*
